In [3]:
import pandas as pd
import os
import concurrent.futures
import os
import requests

In [4]:
def make_dir(d):
    '''Utility for making a directory if not existing.'''
    if not os.path.exists(d):
        os.makedirs(d)        

def get_spe_inter(hic_data, alpha=0.05):
    '''Filter Hi-C data for significant interactions based on the alpha threshold.'''
    hic_spe = hic_data.loc[hic_data['fdr'] < alpha]
    return hic_spe

def get_fold_inputs(spe_df):
    '''Prepare folding input file from the filtered significant interactions.'''    
    spe_out_df = spe_df[['ibp','jbp','fq','i1','j1']]
    spe_out_df['chr'] = "chr1"
    spe_out_df['w'] = [1] * len(spe_out_df)
    result = spe_out_df[['chr','ibp','jbp','fq','w']]
    return result

# load all hic data
all_hic = pd.read_csv('../Data/chr1.27120000.27895000/hic.clean.1/hic.clean.csv.gz')

# call non-random interactions
spe_hic = get_spe_inter(all_hic, alpha=0.05)

# get the folding input file
fold_hic = get_fold_inputs(spe_hic)

# Define the output directory and file path
output_dir = '../Folding_input'
fold_hic_file = 'chr1.27120000.27895000.txt'
fold_hic_path = os.path.join(output_dir, fold_hic_file)
make_dir(output_dir)
fold_hic.to_csv(fold_hic_path, header=None, index=None, sep='\t', mode='a')

/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_55548/2923341863.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spe_out_df['chr'] = "chr1"
/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_55548/2923341863.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spe_out_df['w'] = [1] * len(spe_out_df)


In [4]:
def fetch_chrom_sizes(genome, output_file):
    full_file_path = os.path.join(os.getcwd(), output_file)

    url = f'http://hgdownload.soe.ucsc.edu/goldenPath/{genome}/bigZips/{genome}.chrom.sizes'
    response = requests.get(url)

    if response.status_code == 200:
        with open(full_file_path, 'w') as f:
            for line in response.text.splitlines():
                chrom, size = line.split()
                f.write(f"{chrom}\t{size}\n")
        print(f"Chromosome sizes saved to {full_file_path}")
    else:
        print(f"Failed to fetch chromosome sizes for {genome}")

output_file = 'chromosome_sizes.txt'
fetch_chrom_sizes('hg38', output_file)

Chromosome sizes saved to /Users/siyuanzhao/Documents/GitHub/CS522_Project/Python/chromosome_sizes.txt
